In [7]:
import io
import numpy as np
from pymrmr import mRMR
import pandas as pd
import requests as rq

#for decoder training
from riglib.bmi import accumulator, assist, bmi, clda, extractor, feedback_controllers, goal_calculators, robot_arms, sim_neurons, kfdecoder, ppfdecoder, state_space_models, train
from riglib.bmi.sim_neurons import KalmanEncoder

# use their test case as an example

In [12]:


data_url = 'http://home.penglab.com/proj/mRMR/test_colon_s3.csv'
data = rq.get(data_url).content
data = pd.read_csv(io.StringIO(data.decode('utf-8')))

feats = {
  10: set(['v765', 'v1123', 'v1772', 'v286', 'v467', 'v377', 'v513',
    'v1325', 'v1972', 'v1412']),
  20: set(['v765', 'v1123', 'v1772', 'v286', 'v467', 'v377', 'v513',
    'v1325', 'v1972', 'v1412', 'v1381', 'v897', 'v1671', 'v1582', 'v1423',
    'v317', 'v249', 'v1473', 'v1346', 'v125'])}

for n in feats.keys():
  selected_feats = mRMR(data, 'MIQ', n)
  assert feats[n] == set(selected_feats)

print('confirmed')

confirmed


1. First parameter is a pandas DataFrame (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) containing the input dataset, discretised as defined in the original paper (for ref. see http://home.penglab.com/proj/mRMR/). 
    1. The rows of the dataset are the different samples. The first column is the classification (target) variable for each sample. 
    2. The remaining columns are the different variables (features) which may be selected by the algorithm. (see "Sample Data Sets" at http://home.penglab.com/proj/mRMR/ to download sample dataset to test this algorithm);

2. Second parameter is a string which defines the internal Feature Selection method to use (defined in the original paper): possible values are "MIQ" or "MID";

3.  Third parameter is an integer which defines the number of features that should be selected by the algorithm.

# load features

the following code has been copied and pasted from gen_test_data of decoder_training.ipynb
the idea is to generate cosine tuned spikes from states. 

and use mrmr select useful states.


## generate states

In [2]:
from riglib.bmi.state_space_models import StateSpaceEndptVel2D
ssm = StateSpaceEndptVel2D()


n_samples = 2000

#W = np.identity(N_STATES)
#W = np.zeros((N_STATES,N_STATES))
A, _, W = ssm.get_ssm_matrices()

mean = np.zeros(A.shape[0])
mean[-1] = 1
state_samples = np.random.multivariate_normal(mean, W, n_samples)

state_samples
state_samples.shape

(2000, 7)

## set up encoder

In [3]:
N_NEURONS = 4
N_STATES = 7  # 3 positions and 3 velocities and an offset
# build the observation matrix
sim_C = np.zeros((N_NEURONS, N_STATES))
# control x positive directions
sim_C[0, :] = np.array([0, 0, 0, 1, 0, 0, 0])
sim_C[1, :] = np.array([0, 0, 0, -1, 0, 0, 0])
# control z positive directions
sim_C[2, :] = np.array([0, 0, 0, 0, 0, 1, 0])
sim_C[3, :] = np.array([0, 0, 0, 0, 0, -1, 0])

## train the decoder from the simulated data  

In [4]:
from riglib.bmi.sim_neurons import GenericCosEnc#CLDASimCosEnc

print('SimCosineTunedEnc SSM:', ssm)
encoder = GenericCosEnc(sim_C, 
                         ssm, 
                         return_ts=True, 
                         DT=0.1, 
                         call_ds_rate=6)
encoder.call_ds_rate = 1
sim_C.shape

SimCosineTunedEnc SSM: State space: ['hand_px', 'hand_py', 'hand_pz', 'hand_vx', 'hand_vy', 'hand_vz', 'offset']


(4, 7)

## gen simulated spikes

In [8]:
n_units = sim_C.shape[0]
spike_counts = np.zeros([n_units, n_samples])

for k in range(n_samples):
    spike_counts[:,k] = np.array(encoder(state_samples[k],
                                              mode='counts')).ravel()

kin = state_samples.T
trained_decoder = train.train_KFDecoder_abstract(ssm, 
                                              kin, 
                                              spike_counts, 
                                              encoder.get_units(), 
                                              0.1, 
                                              zscore=False)
trained_decoder.filt.C

{}
end of kwargs
('zscore value: ', False, <class 'bool'>)
no init_zscore


matrix([[ 0.        ,  0.        ,  0.        ,  0.0528346 ,  0.        ,
          0.00285365,  0.11231954],
        [ 0.        ,  0.        ,  0.        , -0.04934332,  0.        ,
          0.00215825,  0.10246379],
        [ 0.        ,  0.        ,  0.        , -0.00209456,  0.        ,
          0.05525108,  0.11845983],
        [ 0.        ,  0.        ,  0.        , -0.00133817,  0.        ,
         -0.04753719,  0.1084568 ]])

# Cast features into feature selection problem
by this I want to transform the variables

## states are targets

In [9]:
print(kin.shape)
#want an easy case, just x y, on the screeen
kin_x_vel = kin[3,:]

kin_x_vel = np.expand_dims(kin_x_vel, axis = 1).T
kin_x_vel.shape

y = kin_x_vel.T
y.shape

(7, 2000)


(2000, 1)

#  firing rates are features

In [10]:
X = spike_counts.T

print(X.shape)

(2000, 4)


# feature selection using regularized linear regression

to show that featuer selection can be done by
dropping features with zero weights  

will copy and paste the stuff from decoder_training.ip

# feature selection with mrmr

In [ ]:
mRMR